In [2]:
import torch
import numpy as np
import ot
from telegram_bot import send_message

In [3]:
#GPU
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [5]:
def wasserstein_dist(a,b,metric, d):
    
    source = a[torch.nonzero(a, as_tuple = False)[:,0], torch.nonzero(a, as_tuple = False)[:,1]].view(torch.nonzero(a, as_tuple = False).size(0)).double()
    target = b[torch.nonzero(b, as_tuple = False)[:,0], torch.nonzero(b, as_tuple = False)[:,1]].view(torch.nonzero(b, as_tuple = False).size(0)).double()
    
    source = source.to(device)
    target = target.to(device)
    
    source = source/torch.sum(source, dtype = torch.double)
    target = target/torch.sum(target, dtype = torch.double)
    
    M = ot.dist(torch.nonzero(a, as_tuple = False).float(), torch.nonzero(b, as_tuple = False).float(), metric = metric).to(device)
    M = M/d
    
    T = ot.emd(source, target, M).to(device)
    W = torch.sum(T*M).to(device)
    
    W = torch.sqrt(W)
    
    return W

In [6]:
data, targets = torch.load('/data/gabrieleb/data/MNIST/validation_eq_s=1000')
data = data.to(device)
d = 2*28*28

In [7]:
#Wasserstein distance 

metric='sqeuclidean'

size = data.size(0)

Wasserstein_dist=torch.zeros(size, size, dtype = torch.double).to(device)

for i in range (size):
    for j in filter(lambda h: h>i, range (size)):
        Wasserstein_dist[i][j] = wasserstein_dist(data[i], data[j], metric, d).to(device)
    if i%5 == 0:
        print(f'Rows done: {(i/size)*100} %')
        await send_message(f'Rows done: {(i/size)*100} %')

Rows done: 0.0 %
Rows done: 0.5 %


CancelledError: 

In [ ]:
torch.save([Wasserstein_dist, targets],'data/MNIST/Wasserstein_dist_validation_eq_s='+str(size))

funzione per estrarre valori: scipy.sqrform